In [1]:
# install packages
!pip install huggingface_hub
!pip install torch 
!pip install accelerate 
!pip install transformers 
!pip install bitsandbytes
!pip install -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-

In [13]:
# load API key in a save way
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_key = user_secrets.get_secret("Hugging Face")

# log into huggingface account with API key
from huggingface_hub import login
login(token = HF_key)

In [7]:
# import libraries
from transformers import pipeline
import os
import transformers
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig)
import torch
import pandas as pd
import bitsandbytes as bnb
import json
import random
import re
import ast
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [5]:
# empty the memory and check if the GPU is available
torch.cuda.empty_cache()
torch.cuda.is_available()

True

In [8]:
# load the sentences and correct labels
all_sentences = []

with open("/kaggle/input/annotations/annotations.json", "r") as f:
    data = json.load(f)

for sentence in data:
    text = sentence["data"]["sentence"]
    labels = []
    results = sentence["annotations"][0]["result"]
    labels = [r["value"]["text"] for r in results]
    
    all_sentences.append({
        "text": text,
        "labels": labels
    })

In [14]:
# specify the models name
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# add the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# quantization options to compress the model to that it fits with the memory
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, 
    bnb_4bit_quant_type = 'nf4', 
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.bfloat16
)

# load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config = quantization_config # with quantization
)

# instantiate a pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True
)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0


In [15]:
# define chat template for normal inference
def compile_ner_prompt(few_shot_examples, test_sentence):
    chat = [
          {"role": "system",
           "content": (
                "You are a helpful assistant that extracts social group mentions from text.\n"
                "Definition of a social group: A social group is a collective of people with common socio-demographic characteristics "
                "(e.g., students, migrants, teachers, women, workers). It can also be formed by shared values or life experiences. "
                "Institutions or institutional groupings do not count as social groups. However, if the sentence refers to the people "
                "of an institution (e.g., 'the patients in the hospital'), this does count as a social group.\n\n"
                "Your task is to extract all social group mentions from a given sentence.\n"
                "Collect all social group mentions into a single list. If there are several group mentions, this list will have several entries.\n"
                "If there are no social group mentions in the sentence, respond with 'None'."
                )
        }
    ]
    # add few-shot examples
    for example in few_shot_examples:
        context = example["text"]
        if example["labels"]:
            answer = str(example["labels"])
        else:
            answer = "None"
        chat.append(
            {"role": "user", "content": f"Sentence: {context}"})
        chat.append({"role": "assistant", "content": answer})
    
    # add the test sentence
    chat.append(
        {"role": "user", "content": f"Sentence: {test_sentence}"})

    return chat

In [16]:
# create some few-shot examples
non_empty_examples = [ex for ex in all_sentences if ex["labels"]]
empty_examples = [ex for ex in all_sentences if not ex["labels"]]
few_shot_examples = random.sample(non_empty_examples, 4) + random.sample(empty_examples, 1)

# create test dataset
split_idx = int(len(non_empty_examples)*0.75)
test_dataset = non_empty_examples[split_idx:] + random.sample(empty_examples, int(len(empty_examples)*0.2))
random.shuffle(test_dataset)

In [17]:
def to_list_or_empty(entry):
    try:
        val = ast.literal_eval(entry)
        if isinstance(val, list):
            return val
        else:
            return []
    except (ValueError, SyntaxError):
        return []

In [19]:
# generate the answers for the normal format and store in a list
gen_answers = []
for i in range(len(test_dataset)):
    sentence = test_dataset[i]["text"]
    prompt = compile_ner_prompt(few_shot_examples, sentence)
    outputs = pipeline(
        prompt,
        max_new_tokens=10,
        pad_token_id=tokenizer.eos_token_id,)
    label = outputs[0]['generated_text'][-1]['content']
    answer_list = to_list_or_empty(label)
    gen_answers.append(answer_list)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [21]:
# evaluate the generated answers
results = []

for idx in range(len(test_dataset)):
    ground_truth = test_dataset[idx]["labels"]
    prediction = gen_answers[idx]
    results.append({"labels": ground_truth,
                    "prediction": prediction})

def evaluate_predictions(results):
    y_true = []
    y_pred = []

    for example in results:
        gold_mentions = set([m.lower().strip() for m in example["labels"]])
        pred_mentions = set([m.lower().strip() for m in example["prediction"]])

        for mention in gold_mentions:
            y_true.append(1)
            y_pred.append(1 if mention in pred_mentions else 0)

        for mention in pred_mentions:
            if mention not in gold_mentions:
                y_true.append(0)
                y_pred.append(1)

    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return precision, recall, f1

p, r, f1 = evaluate_predictions(results)

print(f"Precision: {p:.4f} \nRecall: {r:.4f} \nF1: {f1:.4f}")

Precision: 0.4783 
Recall: 0.5789 
F1: 0.5238


In [23]:
# print some examples
for idx in range(10):
    print(test_dataset[idx]["text"])
    print(results[idx]["labels"])
    print(results[idx]["prediction"])
    print("-"*80)

Lest we forget.
[]
[]
--------------------------------------------------------------------------------
11th December

DAY 11 of my Digital Advent Calendar - I’m featuring Depaul UK

They provide accommodation & support for homeless young people aged 16-25, as well as employment workshops to help with CVs and applying for jobs, colleges & university.
['homeless young people']
[]
--------------------------------------------------------------------------------
I have been working hard to solve a problem with mobile phone signal in Middleton after residents and businesses raised it with me.
['residents']
['residents', 'businesses']
--------------------------------------------------------------------------------
Loved the Turner Gallery & Now Art as well.
[]
[]
--------------------------------------------------------------------------------
Last week, I challenged Minister Diana Johnson over her absolutely shocking U-turn on supporting safe drug consumption rooms - which could save thousand